In [ ]:
!pip install datasets transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00


# Examples: Build Tokenizer From Scratch
Building a tokenizer from scratch is a fundamental step in natural language processing (NLP), enabling the conversion of raw text into numerical data that models can process. The Hugging Face Tokenizers library offers a comprehensive and efficient framework for creating custom tokenizers tailored to specific datasets and applications. This process involves several key components:

- Normalization: Cleaning and standardizing the text, such as removing accents or lowercasing characters.

- Pre-tokenization: Splitting text into preliminary units, often words or subwords.

- Model Training: Learning the tokenization rules from the data, using algorithms like Byte Pair Encoding (BPE), WordPiece, or Unigram.

- Post-processing: Adding special tokens and generating attention masks to prepare the data for model input.

By leveraging these components, the Tokenizers library allows for the construction of efficient and effective tokenizers, facilitating the development of robust NLP models. From Huggingface NLP [tutorial](https://huggingface.co/learn/nlp-course/en/chapter6/8?fw=pt) and [example](https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb)

## Building a wordpiece tokenizer

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")


def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

# with open("wikitext-2.txt", "w", encoding="utf-8") as f:
#     for i in range(len(dataset)):
#         f.write(dataset[i]["text"] + "\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
# tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

# tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
print(tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer."))

# pre_tokenizer = pre_tokenizers.Sequence(
#     [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
# )
# pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

# Train
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# tokenizer.model = models.WordPiece(unk_token="[UNK]")
# tokenizer.train(["wikitext-2.txt"], trainer=trainer)

encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

# Write template
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

# Add decoder
tokenizer.decoder = decoders.WordPiece(prefix="##")
print(tokenizer.decode(encoding.ids))

tokenizer.save("tokenizer.json")
# new_tokenizer = Tokenizer.from_file("tokenizer.json")

from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

hello how are u?
[('Let', (0, 3)), ("'", (3, 4)), ('s', (4, 5)), ('test', (6, 10)), ('my', (11, 13)), ('pre', (14, 17)), ('-', (17, 18)), ('tokenizer', (18, 27)), ('.', (27, 28))]
['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.']
2 3
['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '[SEP]']
['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '...', '[SEP]', 'on', 'a', 'pair', 'of', 'sentences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
let ' s test this tokenizer... on a pair of sentences.


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Deep dive building BPE tokenizer

Here we explore the process of building custom tokenizers using the Hugging Face Tokenizers library in more details. We discuss the components involved, the creation and purpose of configuration files like tokenizer_config.json, the role of chat templates in conversational AI, and the importance of files such as merges.txt in reconstructing your tokenizer's behavior.

### 1. Creating and Saving the Tokenizer

When building your tokenizer, you typically begin by defining the model (for example, BPE) and setting up pre-tokenization and post-processing steps:

- **Pre-tokenization:** This step often involves using strategies like ByteLevel to break text into subwords.
- **Training:** A trainer (e.g., BpeTrainer) builds a vocabulary from your training corpus and applies merge operations.
- **Post-processing and Decoding:** These steps ensure that tokens can be converted back into human-readable text correctly.

After training, saving your tokenizer creates several files. When you wrap your tokenizer using the PreTrainedTokenizerFast class and call the save_pretrained method, the process generates:

- `tokenizer.json`: Contains the core tokenizer configuration and state.
- `tokenizer_config.json`: Stores essential metadata such as the tokenizer’s class, special tokens, and (optionally) the chat template.
- `vocab.json` and `merges.txt`: Specific to BPE tokenizers, these files capture the vocabulary and merge operations.

### 2. The Role of tokenizer_config.json

The tokenizer_config.json file is crucial for the following reasons:

- **Consistency Across Environments:** It saves the tokenizer’s configuration, ensuring that the model behaves the same way when reloaded.
- **Special Tokens and Settings:** Information such as beginning-of-sequence, end-of-sequence tokens, and other configuration details are stored here.
- **Optional Chat Template:** For conversational models, a chat_template can be stored to standardize the format of dialogue inputs.

If tokenizer_config.json is absent, you may encounter several issues:

- **Loading Errors:** AutoTokenizer.from_pretrained might not correctly identify the tokenizer type, resulting in errors.
- **Inconsistencies:** The absence forces reliance on internal defaults, which may lead to discrepancies in behavior across different environments.
- **Compatibility Issues:** Other libraries or platforms expecting this configuration might fail to load the tokenizer correctly.

### 3. Chat Templates in Tokenizers

Many open-sourced models include a chat_template field in their tokenizer_config.json. This template defines how conversational data should be structured, ensuring consistency with the model’s training. A typical chat template might format each message like this:

role: content

#### Setting the Chat Template

To incorporate a chat_template into your tokenizer:

- **Define the Template:** For example, you might use Jinja templating:
  
  {% for message in messages %}{{ message['role'] }}: {{ message['content'] }}
  {% endfor %}

- **Assign the Template:** Since the chat_template attribute belongs to the PreTrainedTokenizerFast class, set it on your wrapped tokenizer instance before saving.

- **Save the Tokenizer:** Use the save_pretrained method to ensure that the chat_template is stored in tokenizer_config.json.

#### Testing Your Chat Template

It is essential to test your chat template by:

- Creating sample conversation data (a list of messages, each with 'role' and 'content').
- Applying the chat_template to format the conversation.
- Tokenizing the formatted input and passing it to your model to observe the outputs.

Aligning your training data with the same template is also advisable, so the model learns the expected structure for conversational inputs.

### 4. Understanding the merges.txt File

When you save a tokenizer model, you may notice a merges.txt file. This file is critical for Byte Pair Encoding (BPE) tokenizers:

- **Purpose:** During BPE training, the tokenizer identifies frequent token pairs and merges them to form new tokens. Each merge operation is recorded in the merges.txt file.
- **Structure:** Each line in merges.txt represents a merge operation, and the order reflects the sequence in which these merges occurred.
- **Usage:** When loading a BPE tokenizer, both vocab.json and merges.txt are necessary to accurately reconstruct the tokenization logic.

For instance, using the ByteLevelBPETokenizer class requires you to provide paths to both vocab.json and merges.txt to ensure that the same tokenization behavior is applied.

```python

from tokenizers import ByteLevelBPETokenizer

# Paths to the vocab and merges files
vocab_path = "path/to/vocab.json"
merges_path = "path/to/merges.txt"

# Initialize the tokenizer
tokenizer = ByteLevelBPETokenizer(
    vocab_file=vocab_path,
    merges_file=merges_path
)

# Encode a sample text
encoded = tokenizer.encode("Sample text to tokenize.")
print(encoded.tokens)

```


In [ ]:
from transformers import PreTrainedTokenizerFast
import os

# Initialize your tokenizer (this could be from a pre-trained model or a custom tokenizer)
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

# Wrap the tokenizer with PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)

# Define the chat template
chat_template = "{% for message in messages %}{{ message['role'] }}: {{ message['content'] }}\n{% endfor %}"
# Set the chat template on the wrapped tokenizer
wrapped_tokenizer.chat_template = chat_template

# Save the tokenizer, which includes the tokenizer_config.json
tokenizer_dir = "model/minillm_tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True)
wrapped_tokenizer.save_pretrained(tokenizer_dir)
tokenizer.model.save(tokenizer_dir)

['model/minillm_tokenizer/vocab.json', 'model/minillm_tokenizer/merges.txt']

[Instructions](https://huggingface.co/docs/transformers/main/en/chat_templating) using the chat template

In [ ]:
from transformers import AutoTokenizer

# Load your tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

# Define sample conversation
messages = [
    {"role": "user", "content": "Hello! How are you?"},
    {"role": "assistant", "content": "I'm fine, thank you! How can I assist you today?"}
]

# Apply chat template
formatted_input = tokenizer.apply_chat_template(messages)

# Train MiniLM Tokenizers Using Qwen2.5 as a Template

Hugging Face provides powerful tools to train new tokenizers from your own corpus. Unlike model training, which relies on stochastic optimization, tokenizer training is a deterministic statistical process that identifies the most frequent subword units in your dataset. With the `train_new_from_iterator()` method available in the fast tokenizers, you can efficiently build a custom vocabulary tailored to your domain. For instance, when adapting a tokenizer for a specific mix of Chinese and English texts, you might adjust the vocabulary size and tokenization rules to optimize both compression and semantic fidelity.

See details in `train_tokenizer.py`

## Introduciton of Qwen tokenizer


The Qwen2.5 tokenizer is part of the latest series of Qwen large language models. It exemplifies modern tokenization practices by combining:

- **Byte-level BPE:** Qwen2.5 uses a byte-level subword algorithm on UTF-8 encoded texts. This ensures that no word goes unseen and that even rare characters in languages like Chinese and English are tokenized effectively.
- **Large Vocabulary and Control Tokens:** With a vocabulary exceeding 150,000 tokens, Qwen2.5 is designed to handle over 29 languages. In addition, it incorporates control tokens such as `<|endoftext|>`, `<|im_start|>`, and `<|im_end|>` to manage chat formatting and document boundaries.
- **Extended Context and Generation Capabilities:** The model supports context lengths of up to 128K tokens and can generate outputs of up to 8K tokens, making it highly capable for long-form content and detailed conversations.

These design choices not only provide robust multilingual support but also optimize the tokenization process to minimize unknown tokens and maintain semantic consistency.

See more in the paper: [Qwen](https://arxiv.org/pdf/2309.16609), [Qwen2](https://arxiv.org/pdf/2407.10671) and [Qwen2.5](https://arxiv.org/pdf/2412.15115)

## Train MiniLM tokenizer

In [ ]:
from transformers import AutoTokenizer, Qwen2TokenizerFast

# old_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
# Qwen support fast tokenizer so we can use train_new_from_iterator
old_tokenizer = Qwen2TokenizerFast.from_pretrained("Qwen/Qwen2.5-0.5B")
assert old_tokenizer.is_fast
tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus, 52000)

tokenizer.save_pretrained("code-search-net-tokenizer")

In [ ]:
# !apt install git-lfs

In [ ]:
from huggingface_hub import notebook_login

notebook_login()
tokenizer.push_to_hub("my-new-shiny-tokenizer")